In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
import os
qprc_path = os.path.abspath(os.path.join('..'))
if qprc_path not in sys.path:
    sys.path.append(qprc_path)

#from utils.plots import *
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np
import numpy.ma as ma
import csv
from scipy.stats import randint
from imblearn.over_sampling import SMOTE

import seaborn as sns
from kernelfuzzy.kernels import KBFkernelSymmetric
from  kernelfuzzy.fuzzyset import FuzzySet
from kernelfuzzy.fuzzysystem import *
from sklearn.datasets.samples_generator import make_classification
from sklearn.datasets import make_moons, make_circles,make_blobs,load_digits
from sklearn.svm import SVC,NuSVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.metrics import accuracy_score
import skfuzzy as fuzz # for FCM
import pickle
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, StratifiedKFold
from c45 import C45
from sklearn.preprocessing import StandardScaler, MinMaxScaler,RobustScaler, Normalizer,QuantileTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

from fylearn.nfpc import FuzzyPatternClassifier
from fylearn.garules import MultimodalEvolutionaryClassifier
from fylearn.fpt import FuzzyPatternTreeTopDownClassifier

from fylearn.nfpc import FuzzyPatternClassifier  #base class for fuzzy pattern classifiers (see parameters)
from fylearn.garules import MultimodalEvolutionaryClassifier #learns rules using genetic algorithm
from fylearn.fpt import FuzzyPatternTreeTopDownClassifier #builds fuzzy pattern trees using top-down method.
from fylearn.frr import FuzzyReductionRuleClassifier # based on learning membership functions from min/max.
from fylearn.fpcga import FuzzyPatternClassifierGA # optimizes membership functions globally.
from fylearn.fpt import FuzzyPatternTreeClassifier # builds fuzzy pattern trees using bottom-up method.
from experimentscripts.functions_experiment_attribute_noise import * 

In [2]:
def main():
    experiment_description={'noise_level':[0,5,10,15,20],
                                'type_noise' :'nn', #options are nn, nc, cn
                                'dataset_name':'iris',
                                'n_iter':10,
                                'output_dir':'../experiments/attribute_noise',
                                'n_splits_outter':5,
                                'n_splits_inner':5,
                                'classifier':'NSFS_KBF_symmetric'
                               }
    do_experiments(experiment_description)
    
    '''
    list_classifiers=['NSFS_NS','NSFS_KBF_symmetric',
                      'C4.5','bagC45',
                      'svmRBF','knn',
                     'FuzzyPatternClassifier','MultimodalEvolutionaryClassifier',
                     'FuzzyPatternTreeTopDownClassifier','FuzzyReductionRuleClassifier',
                     'FuzzyPatternClassifierGA','FuzzyPatternTreeClassifier','lr','rf','mlp','sgd'
                     ]
    
    list_datasets=['iris','wine','sonar','glass','heart',
                   'ecoli','ionosphere','wdbc','pima','contraceptive',
                   'yeast','segment','spambase','page-blocks','satimage',
                   'thyroid','ring','twonorm','penbased']
    
    for classifier in list_classifiers:
        experiment_description={'noise_level':[0,5,10,15,20],
                                'type_noise' :'nn', #options are nn, nc, cn
                                'dataset_name':'iris',
                                'n_iter':10,
                                'output_dir':'../experiments/attribute_noise',
                                'n_splits_outter':5,
                                'n_splits_inner':5,
                                #'classifier':list_classifiers[3],
                                'classifier':classifier
                               }
        do_experiments(experiment_description)
       ''' 
if __name__ == "__main__":
    main()
    
#or python experiments_attribute_noise.py -d iris -c knn -i 10 -tn nn -o ../experiments/attribute_noise -no 5 -ni 5
#wine 0 [0.76744186 0.65116279 0.97674419 1.         0.88095238]
# noise =  5 [1.         0.97674419 0.95348837 0.95238095 0.85714286]
# noise =  10 [0.95348837 0.93023256 0.97674419 0.97619048 0.92857143]

../data/attributenoise/nn/attribute_noise_0_nn/wine-0an-nn/wine-5an-nc-5-2tst.dat
../data/attributenoise/nn/attribute_noise_0_nn/wine-0an-nn/wine-5an-cn-5-1tra.dat
../data/attributenoise/nn/attribute_noise_0_nn/wine-0an-nn/wine-5an-cn-5-3tra.dat
../data/attributenoise/nn/attribute_noise_0_nn/wine-0an-nn/wine-5an-nc-5-4tst.dat
../data/attributenoise/nn/attribute_noise_0_nn/wine-0an-nn/wine-5an-cn-5-5tra.dat
../data/attributenoise/nn/attribute_noise_0_nn/wine-0an-nn/wine-5an-nc-5-3tst.dat
../data/attributenoise/nn/attribute_noise_0_nn/wine-0an-nn/wine-5an-cn-5-2tra.dat
../data/attributenoise/nn/attribute_noise_0_nn/wine-0an-nn/wine-5an-nc-5-1tst.dat
../data/attributenoise/nn/attribute_noise_0_nn/wine-0an-nn/wine-5an-nc-5-5tst.dat
../data/attributenoise/nn/attribute_noise_0_nn/wine-0an-nn/wine-5an-cn-5-4tra.dat
before eliminating duplicates  shape (890, 14)
after eliminating duplicates  shape (178, 14)
NSFS_KBF_symmetric , noise =  0


KeyboardInterrupt: 

## Experiment Analysis

In [ ]:
def plot_sns(df,list_classifiers):
    df_plot= df[df.classifier.isin(list_classifiers)]
    sns.set(style="darkgrid")
    #g=sns.lineplot(x="noise", y="accuracy",
    #               hue="classifier", hue_order=list_classifiers,
    #               style="classifier",style_order=list_classifiers,
    #               data=df_plot)
    g=sns.lineplot(x="noise", y="accuracy", hue="classifier",hue_order=list_classifiers,
                   style="classifier",style_order=list_classifiers,
                   err_style="band", 
                   ci='sd', 
                   estimator="median", 
                   data=df_plot)
    g.set_xticks([0,5,10,15,20])

    # Put the legend out of the figure
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

def plot_sns_pointplot(df,list_classifiers):
    df_plot= df[df.classifier.isin(list_classifiers)]
    sns.set(style="darkgrid")
    #sns.lineplot(x="dimension", y="accuracy",
    #             hue="classifier", hue_order=list_classifiers,
    #             style="classifier",style_order=list_classifiers,
    #             data=df_plot)
    sns.pointplot(x="noise", y="accuracy", data=df_plot,
                 hue="classifier", hue_order=list_classifiers,
                 style="classifier",style_order=list_classifiers,
                 estimator=np.median,
                 linestyles="--",
                 ci=None)
    # Put the legend out of the figure
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    
#PLOTS NESTED CV
import os
from functools import reduce
import seaborn as sns

#reading all the results starting with 'results'
list_classifiers=['NSFS_NS','NSFS_KBF_symmetric',
                      'C4.5','bagC45',
                      'svmRBF','knn',
                     'FuzzyPatternClassifier','MultimodalEvolutionaryClassifier',
                     'FuzzyPatternTreeTopDownClassifier','FuzzyReductionRuleClassifier',
                     'FuzzyPatternClassifierGA','FuzzyPatternTreeClassifier','lr','rf','mlp','sgd'
                 ]

path = '../experiments/attribute_noise'
#joining files
files = [filename for filename in os.listdir(path) if filename.startswith("nn")]
l_df= [pd.read_csv(path+'/'+file,index_col=0)  for file in files]
df_results_non_nested = reduce(lambda left,right: pd.merge(left,right,on=['noise','trial'],left_index=True), l_df)
#print(df_results_non_nested)
#melt
df = df_results_non_nested.copy()
df = pd.melt(df, id_vars=['trial','noise'], value_vars=list_classifiers,
             var_name='classifier', value_name='accuracy')
df = df.sort_values(by='noise')
    
#ploting NSFS classifiers vs classical classifiers I
#-------------------------------------------------
#plt.figure(figsize=(20, 10))
list_classifiers=['NSFS_NS','NSFS_KBF_symmetric','lr','svmRBF','knn',]
plot_sns_pointplot(df,list_classifiers)
list_classifiers=['NSFS_NS','NSFS_KBF_symmetric','rf','mlp','sgd']
plot_sns_pointplot(df,list_classifiers)

list_classifiers=['NSFS_NS','NSFS_KBF_symmetric','C4.5','bagC45']
plot_sns_pointplot(df,list_classifiers)

list_classifiers=['NSFS_NS','NSFS_KBF_symmetric','FuzzyPatternClassifier',
                 'MultimodalEvolutionaryClassifier',
                     'FuzzyPatternTreeTopDownClassifier']
plot_sns_pointplot(df,list_classifiers)

list_classifiers=['NSFS_NS','NSFS_KBF_symmetric','FuzzyReductionRuleClassifier',
                     'FuzzyPatternClassifierGA','FuzzyPatternTreeClassifier']
plot_sns_pointplot(df,list_classifiers)

# sklearn bug, see github issues

In [ ]:
from sklearn.datasets import load_boston,load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
import numpy as np


data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'])
inner_cv = KFold(n_splits=2, shuffle=True, random_state=0)
outer_cv = KFold(n_splits=2, shuffle=True, random_state=0)

#######
scalers_to_test = [MinMaxScaler(), None]
pipe_KBF_symmetric = Pipeline([('scaler', MinMaxScaler()),
                               ('Fuzzifier', NonSingletonFuzzifier(constant_std=True)),
                               ('kernel', KBFkernelSymmetric()),
                               ('svm', SVC())])

cv_params = {'scaler': scalers_to_test,
             'Fuzzifier__std_proportion': np.arange(0.01, np.std(X_train), 0.1),
             'kernel__param': 2.0 ** np.arange(-20, 1),
             'svm__kernel': ['precomputed'],
             'svm__C': 2.0 ** np.arange(-15, 15)}

clf = RandomizedSearchCV(pipe_KBF_symmetric, cv_params, cv=inner_cv, verbose=1, n_jobs=-1, n_iter=3)
nested_score = cross_val_score(clf, X=X_train, y=y_train, cv=outer_cv,n_jobs=-1)
print(nested_score)

#

cv_params = [dict([('scaler',[scalers_to_test[0]]),
                   ('Fuzzifier__std_proportion', np.arange(0.01, np.std(MinMaxScaler().fit(X_train).transform(X_train)), 0.1)),
                   ('kernel__param', 2.0 ** np.arange(-20, 1)),
                   ('svm__kernel', ['precomputed']),
                   ('svm__C', 2.0 ** np.arange(-15, 15))]),
             dict([('scaler',[scalers_to_test[1]]),
                   ('Fuzzifier__std_proportion', np.arange(0.01, np.std(X_train), 0.1)),
                   ('kernel__param', 2.0 ** np.arange(-20, 1)),
                   ('svm__kernel', ['precomputed']),
                   ('svm__C', 2.0 ** np.arange(-15, 15))])
                    ]

cv_params = [
        {'scaler': [scalers_to_test[0]],
         'Fuzzifier__std_proportion': np.arange(0.01, np.std(MinMaxScaler().fit(X_train).transform(X_train)), 0.1),
         'kernel__param':  2.0 ** np.arange(-20, 1),
         'svm__kernel': ['precomputed'],
         'svm__C': 2.0 ** np.arange(-15, 15)},
        {'scaler': [scalers_to_test[1]],
         'Fuzzifier__std_proportion': np.arange(0.01, np.std(MinMaxScaler().fit(X_train).transform(X_train)), 0.1),
         'kernel__param':  2.0 ** np.arange(-20, 1),
         'svm__kernel': ['precomputed'],
         'svm__C': 2.0 ** np.arange(-15, 15)},
        ]
clf = RandomizedSearchCV(pipe_KBF_symmetric, cv_params, cv=inner_cv, verbose=1, n_jobs=-1, n_iter=3)
nested_score = cross_val_score(clf, X=X_train, y=y_train, cv=outer_cv,n_jobs=-1)
print(nested_score)

## posted on github

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import  cross_val_score, KFold,RandomizedSearchCV

import numpy as np

iris = load_iris()
X = iris.data 
y = iris.target
pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC())
        ])

params = {'scaler': [StandardScaler(), MinMaxScaler(),RobustScaler(),None],
          'svm__C': 2.0 ** np.arange(-15, 15)}
#nested cv
inner_cv = KFold(n_splits=3, shuffle=True, random_state=0)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=0)
clf = RandomizedSearchCV(pipe, params, cv=inner_cv, verbose=1, n_jobs=-1, n_iter=3)
nested_score = cross_val_score(clf, X=X, y=y, cv=outer_cv,n_jobs=-1)
print(nested_score)


In [ ]:
from sklearn.datasets import load_iris
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
iris = load_iris()
print(iris.data)
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
print('duplicates :{}'.format(df[df.duplicated(keep=False)]))

print(df.head())
print(df.drop_duplicates().reset_index(drop=True).shape)
print(df.drop_duplicates().shape)
print('len df', df.shape)
print('duplicates', pd.DataFrame.duplicated(df))


## generating strings for experiments


In [47]:
#python experiments_attribute_noise.py -d iris -c knn -i 10 -tn nn -o ../experiments/attribute_noise -no 5 -ni 5
list_classifiers=['NSFS_NS','NSFS_KBF_symmetric',
                      'C4.5','bagC45',
                      'svmRBF','knn',
                     'FuzzyPatternClassifier','MultimodalEvolutionaryClassifier',
                     'FuzzyPatternTreeTopDownClassifier','FuzzyReductionRuleClassifier',
                     'FuzzyPatternClassifierGA','FuzzyPatternTreeClassifier','lr','rf','mlp','sgd'
                     ]

list_datasets=['iris','wine','sonar','glass','heart',
                   'ecoli','ionosphere','wdbc','pima','contraceptive',
                   'yeast','segment','spambase','page-blocks','satimage',
                   'thyroid','ring','twonorm','penbased']

list_classifiers=['bagC45']

list_datasets=['page-blocks']

str_exp='export PATH=$PATH:/home/jorjasso/anaconda3/bin && source activate fuzzyKernels '

l_exp=[]
for dataset in list_datasets:
    for classifier in list_classifiers:
        l_exp.append(' && screen -d -m python experiments_attribute_noise.py -d '+dataset+' -c '+classifier+' -i 10 -tn nn -o ../experiments/attribute_noise -no 5 -ni 5')

print(len(l_exp))
print(str_exp+''.join(l_exp))

#verificar NSFS_NS con 'yeast','segment','spambase','page-blocks','satimage', puchkin, tolstoi, dostoievski, hulk1 hulk4

1
export PATH=$PATH:/home/jorjasso/anaconda3/bin && source activate fuzzyKernels  && screen -d -m python experiments_attribute_noise.py -d page-blocks -c bagC45 -i 10 -tn nn -o ../experiments/attribute_noise -no 5 -ni 5


# optimizing code for kernel estimation: testing times


In [20]:
import numpy as np
import numba as nb
from numba import njit, prange
import timeit

def gram_matrix_nonsingleton_gaussian_kernel_njit(mean_X: np.ndarray,
                                                  mean_Y: np.ndarray,
                                                  sigma_X: np.ndarray,
                                                  sigma_Y: np.ndarray,
                                                  param: float) -> np.ndarray:
    '''

        Calculates the Gram matrix using the nonsingleton Gaussian kernel on fuzzy sets
        optimized for Numba

        Input:
            mean_X, sigma_X:     matrix of parameters for fuzzy data X (Type: FuzzyData)
            mean_Y, sigma_Y:     matrix of parameters for fuzzy data Y (Type: FuzzyData)
            param:                  (Type: float) kernel parameter

        Output:
            (Type: numpy.ndarray) kernel matrix

        '''

    gram_matrix = np.zeros((mean_X.shape[0], mean_Y.shape[0]))


    if mean_X is mean_Y:  # use symmetry property
        N = mean_X.shape[0]

        #for i in range(0, N):
        for i in prange(N):
            for j in range(i, N):

                value = 1
                for mean_x, mean_y, sigma_x, sigma_y in zip(mean_X[i, :], mean_Y[j, :], sigma_X[i, :], sigma_Y[j, :]):
                    value = value * (np.exp(-0.5 * param * (mean_x - mean_y) ** 2 / (sigma_x ** 2 + sigma_y ** 2)))

                gram_matrix[i, j] = value
                gram_matrix[j, i] = value
        # tikonov regularization
        gram_matrix = gram_matrix + np.eye(N) * np.nextafter(0, 1)


    else:  # X and Y are different
        N = mean_X.shape[0]
        M = mean_Y.shape[0]

        #for i in range(0, N):
        for i in prange(N):
            for j in range(0, M):

                value = 1
                for mean_x, mean_y, sigma_x, sigma_y in zip(mean_X[i, :], mean_Y[j, :], sigma_X[i, :], sigma_Y[j, :]):
                    value = value * (np.exp(-0.5 * param * (mean_x - mean_y) ** 2 / (sigma_x ** 2 + sigma_y ** 2)))

                gram_matrix[i, j] = value

    return gram_matrix


func2_nb = nb.jit(nopython=True, nogil=True, parallel=False)(gram_matrix_nonsingleton_gaussian_kernel_njit)
func2_nb_p = nb.jit(nopython=True, nogil=True, parallel=True)(gram_matrix_nonsingleton_gaussian_kernel_njit)

val=50
mean_X=mean_Y=sigma_X=sigma_Y=np.random.random_sample([val,val])


#%timeit -n 10 gram_matrix_nonsingleton_gaussian_kernel_njit(mean_X,mean_Y,sigma_X,sigma_Y,1)
print('---')
#%timeit -n 10 func2_nb(mean_X,mean_Y,sigma_X,sigma_Y,1)
print('---')
#%timeit -n 10 func2_nb_p(mean_X,mean_Y,sigma_X,sigma_Y,1)
print('---')
gram_matrix_nonsingleton_gaussian_kernel_njit(mean_X,mean_Y,sigma_X,sigma_Y,1)
#func2_nb(mean_X,mean_Y,sigma_X,sigma_Y,1)
func2_nb_p(mean_X,mean_Y,sigma_X,sigma_Y,1)


---
---
---


AssertionError: Failed at nopython (convert to parfors)


In [18]:
import numpy as np
import numba as nb
from numba import njit, prange
import timeit

def gram_matrix_nonsingleton_gaussian_kernel_njit(mean_X: np.ndarray,
                                                  mean_Y: np.ndarray,
                                                  sigma_X: np.ndarray,
                                                  sigma_Y: np.ndarray,
                                                  param: float) -> np.ndarray:
    '''

        Calculates the Gram matrix using the nonsingleton Gaussian kernel on fuzzy sets
        optimized for Numba

        Input:
            mean_X, sigma_X:     matrix of parameters for fuzzy data X (Type: FuzzyData)
            mean_Y, sigma_Y:     matrix of parameters for fuzzy data Y (Type: FuzzyData)
            param:                  (Type: float) kernel parameter

        Output:
            (Type: numpy.ndarray) kernel matrix

        '''

    gram_matrix = np.zeros((mean_X.shape[0], mean_Y.shape[0]))
    N = mean_X.shape[0]
    for i in prange(N):
        for j in range(i, N):
            value = i
            gram_matrix[i, j] = value
            gram_matrix[j, i] = value
    #

    return gram_matrix 


func2_nb = nb.jit(nopython=True, nogil=True, parallel=False)(gram_matrix_nonsingleton_gaussian_kernel_njit)
func2_nb_p = nb.jit(nopython=True, nogil=True, parallel=True)(gram_matrix_nonsingleton_gaussian_kernel_njit)

val=50
mean_X=mean_Y=sigma_X=sigma_Y=np.random.random_sample([val,val])


#%timeit -n 10 gram_matrix_nonsingleton_gaussian_kernel_njit(mean_X,mean_Y,sigma_X,sigma_Y,1)
print('---')
#%timeit -n 10 func2_nb(mean_X,mean_Y,sigma_X,sigma_Y,1)
#%timeit -n 10 func2_nb_p(mean_X,mean_Y,sigma_X,sigma_Y,1)
print('---')
gram_matrix_nonsingleton_gaussian_kernel_njit(mean_X,mean_Y,sigma_X,sigma_Y,1)
#func2_nb(mean_X,mean_Y,sigma_X,sigma_Y,1)
print('---')
func2_nb_p(mean_X,mean_Y,sigma_X,sigma_Y,1)


---
---
---


array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   1.,   1., ...,   1.,   1.,   1.],
       [  0.,   1.,   2., ...,   2.,   2.,   2.],
       ..., 
       [  0.,   1.,   2., ...,  47.,  47.,  47.],
       [  0.,   1.,   2., ...,  47.,  48.,  48.],
       [  0.,   1.,   2., ...,  47.,  48.,  49.]])